# Počasí vs. realita – Transformace všeho

Nejdřív musím naimportovat všechny knihovny 🐼

In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import re

## 1. Čištění CHMI_TEPLOTA

In [2]:
df_chmi_temp = pd.read_csv("in/tables/chmu_tab_teplota.csv")

Potřebuji najít prázdná místa a nahradit je NaN. Prázdná místa ale tady nejsou normální mezery, ale pevné mezery.  
Rada od Toma: Místo toho, abych hledala divný mezery (což by skrze unescape a normalize také šlo), tak najdu hodnoty, které mě naopak zajímají - tedy ty, které sedí do regexu
```
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import re
import numpy as np
​
from html import unescape
from unicodedata import normalize
​
​
df_chmi_temp = pd.read_csv("chmu_tab_teplota.csv")
​
# Matchne optional plus nebo minus ([\+-]?),
# za nim alespon 1 cislici (\d+),
# za ni muze byt (?:nejaka zrudnost)?
# bud desetinna tecka nebo carka ([\.,])
# a alespon 1 cislice (\d+).
# \+ je escapovani plusu aby znamenalo doslova znak plus, a ne kvantifikator "alespon jednou"
# (nejaka zrudnost) je capturing group -- ziska mi co se uvnitr matchlo,
# (?:nejaka zrudnost) je non-capturing group -- slouzi jen k uzavorkovani, neziska vnitrek
# \. je escapovani tecky aby znamenala doslova znak tecka, a ne wildcard pro cokoliv
TEMP_REGEX = re.compile(r"([\+-]?\d+(?:[\.,]\d+)?)")
​
def chmitemptofloat(cell):
    # unescape z HTML zapisu vyrobi unicode znak (tedy z &nbsp; vyrobi zobrazitelnou pevnou mezeru),
    # normalize vrati kanonickou formu unicode znaku (tedy z pevne mezery klasickou mezeru)
    # ta forma NFKC rika jak se to ma konkretne normalizovat, spis se to tyka unicode znaku
    # ktere sestavaji z vic subznaku, tohle by nejdriv rozlozilo ten velky znak (treba
    # C s ocaskem na C a ocasek zvlast), normalizovalo kazdy zvlast a pak zase slozilo vysledek
    decoded = normalize("NFKC", unescape(cell))
    match = TEMP_REGEX.match(decoded)
​
    if match:
        temp = float(match.group(1).replace(",", "."))
    else:
        temp = float("NaN")
​
    return temp
​
df_chmi_temp["Průměrnáteplota"] = df_chmi_temp["Průměrnáteplota"].apply(chmitemptofloat)
​
replaced = df_chmi_temp[df_chmi_temp["Průměrnáteplota"].isna()]
print(replaced)
```

**Musím ze všech sloupců s teplotami udělat FLOAT. (oříznout °C a vyměnit , za .) Z date_stamp musím udělat datetime. Na Relativní vlhkost prdím, protože ji stejně nebudu potřebovat. Stejně jako aktuální teplota (Teplotavzduchu), teplota rosného bodu etc...**

In [3]:
def chmitemptofloat(cell):
    TEMP_REGEX = re.compile(r"([\+-]?\d+(?:[\.,]\d+)?)")
    match = TEMP_REGEX.match(cell)
    if match:
        temp = float(match.group(1).replace(",", "."))
    else:
        temp = float("NaN")
    return temp
    
df_chmi_temp["Prumernateplota"] = df_chmi_temp["Prumernateplota"].apply(chmitemptofloat)
df_chmi_temp["Maximalniteplota"] = df_chmi_temp["Maximalniteplota"].apply(chmitemptofloat)
df_chmi_temp["Minimalniteplota"] = df_chmi_temp["Minimalniteplota"].apply(chmitemptofloat)
df_chmi_temp["date_stamp"] = pd.to_datetime(df_chmi_temp["date_stamp"], format = "%d/%m/%Y %H:%M:%S")

Chci vybrat jen ty řádky, kde date_stamp obsahuje pouze 19:00:00 nebo 20:00:00 (to se stalo, když se přetočil čas). Zbylé časy jsou záznamy, kdy jsem si to zkoušela nanečisto apod. 

In [4]:
df_chmi_temp = df_chmi_temp[(df_chmi_temp["date_stamp"].dt.time.astype(str) == "19:00:00") | (df_chmi_temp["date_stamp"].dt.time.astype(str) == "20:00:00")]

Vezmu si jen sloupce, které mi k něčemu jsou a chytře je pojmenuji. (Tedy vyhodím ty, co jsou k ničemu a zbylé chytře pojmenuji.)

In [5]:
df_chmi_temp = df_chmi_temp.drop(columns=['Teplotavzduchu', 'Teplotarosneho_bodu', 'Relativnivlhkost', 'Minimalniprizemni_teplota'])

In [6]:
df_chmi_temp = df_chmi_temp.rename(columns={"Stanice": "location", "Prumernateplota": "chmi_temp_avg", "Maximalniteplota": "chmi_temp_max", "Minimalniteplota": "chmi_temp_min"})

Ještě tam přidám sloupec, pomocí kterého budu moci joinovat tabulky - datum, pro který den je měření platné. ČHMÚ vždy uvádí průmerné hodnoty za uplynulý den (tedy od včera 6:00 UTC do dnes 6:00 UTC). Nazvu si to forecast_date i když to není forecast, ale podle toho budu na sebe matchovat jednotlivé tabulky.

In [7]:
df_chmi_temp["forecast_date"] = (df_chmi_temp["date_stamp"] - datetime.timedelta(days = 1)).dt.strftime('%Y-%m-%d')

## 2. Čištění CHMI_SRAZKY

In [8]:
df_chmi_rain = pd.read_csv("in/tables/out-tables-chmu_tab_srazky-csv.csv")

Tentokrát na to půjdu chytřeji. Beru jen to, co se mi hodí, a dám tomu chytré názvy.

In [9]:
df_chmi_rain = df_chmi_rain.drop(columns=['Srazky_za_posledni_hodinu', 'Srazky_za_12_hodin', 'Celkova_snehova_pokryvka_cm'])

In [10]:
df_chmi_rain = df_chmi_rain.rename(columns={"Stanice": "location", "Srazky_za_24_hodin": "chmi_rain", "Novy_snih_cm": "chmi_snow"})

Udělám ze str float, vyhodím jednotky a z prázdných míst udělám NaN. Tomáš mi ten regex napsal tak hezky, že na to asi půjde použít pořád stejná funkce jako v případě teplot.

In [11]:
df_chmi_rain["chmi_rain"] = df_chmi_rain["chmi_rain"].apply(chmitemptofloat)
df_chmi_rain["chmi_snow"] = df_chmi_rain["chmi_snow"].apply(chmitemptofloat)
df_chmi_rain["date_stamp"] = pd.to_datetime(df_chmi_rain["date_stamp"], format = "%d/%m/%Y %H:%M:%S")

Chci vybrat jen ty řádky, kde date_stamp obsahuje pouze 19:00:00 nebo 20:00:00 (to se stalo, když se přetočil čas). Zbylé časy jsou záznamy, kdy jsem si to zkoušela nanečisto apod. 

In [12]:
df_chmi_rain = df_chmi_rain[(df_chmi_rain["date_stamp"].dt.time.astype(str) == "19:00:00") | (df_chmi_rain["date_stamp"].dt.time.astype(str) == "20:00:00")]

In [13]:
df_chmi_rain["forecast_date"] = (df_chmi_rain["date_stamp"] - datetime.timedelta(days = 1)).dt.strftime('%Y-%m-%d')

Přidávám kvalitativní sloupeček, jestli pršelo nebo ne. (Což zanedbává neměřitelné srážky...uf..)

In [14]:
df_chmi_rain.loc[df_chmi_rain["chmi_rain"] > 0, "chmi_rain_bool"] = True
df_chmi_rain.loc[df_chmi_rain["chmi_rain"] == 0, "chmi_rain_bool"] = False

## 3. WEATHER.COM (z apify)

In [15]:
df_weathercom = pd.read_csv("in/tables/weathercom_tab-csv.csv")

Vyhazuju sloupečky, co nepotřebuju.

In [16]:
df_weathercom = df_weathercom.drop(columns=['city', 'country', 'humidity', 'locationId', 'state', 'windDirection', 'windSpeed', 'zipCode'])

Chci vybrat jen ty řádky, kde date_stamp obsahuje pouze 19:00:00 nebo 20:00:00 (to se stalo, když se přetočil čas). Zbylé časy jsou záznamy, kdy jsem si to zkoušela nanečisto apod. 

In [17]:
df_weathercom["date_stamp"] = pd.to_datetime(df_weathercom["date_stamp"])
df_weathercom = df_weathercom[(df_weathercom["date_stamp"].dt.time.astype(str) == "19:00:00") | (df_weathercom["date_stamp"].dt.time.astype(str) == "20:00:00")]

Zjistila jsem, že pro to, abych poznala, na který den je předpověď mi stačí datum bez času. Čas tedy z položky time stripuji (neb jsem byla líná vvymýšlet cokoliv sofistikovanějšího) a převádím na datum. Nutno podotknout, že weathercom je ukázněné a data jsou vždy od 6:00 do 6:00, takže vše snad sedí, jak má. 

In [18]:
def todt(time):
    time = str(time)
    time = time.rstrip(time[-12:])
    time = pd.to_datetime(time)
    return time
    
df_weathercom["time"] = df_weathercom["time"].apply(todt)

Další úkol je udělat z 'temperature' dva sloupečky pro maximální a minimální teplotu. Dělám z nich float, aby byly dobře porovnatelné s daty z čhmú. Ale pravdou je, že weathercom dodává jen celá čísla. Asi by se na to mělo myslet v interpretaci výsledků. 

In [19]:
def temptomax(temperature):
    TEMP_REGEX2 = re.compile(r"([\+-]?\d+)(?:[\/])([\+-]?.+)")
    match = TEMP_REGEX2.match(temperature)
    if match:
        temp = float(match.group(1))
    else:
        temp = float("NaN")
    return temp

def temptomin(temperature):
    TEMP_REGEX2 = re.compile(r"([\+-]?.+)(?:[\/])([\+-]?\d+)")
    match = TEMP_REGEX2.match(temperature)
    if match:
        temp = float(match.group(2))
    else:
        temp = float("NaN")
    return temp
    
df_weathercom["wc_temp_max"] = df_weathercom["temperature"].apply(temptomax)
df_weathercom["wc_temp_min"] = df_weathercom["temperature"].apply(temptomin)

Původního sloupce se už tedy můžu zbavit.

In [20]:
df_weathercom = df_weathercom.drop(columns=['temperature'])

Zde ze sloupce 'forecast' tahám slovíčka, která by měla značit srážky a zaznamenávám, jestli hlásili déšť nebo ne. 

In [21]:
df_weathercom.loc[df_weathercom['forecast'].str.contains('Rain|Showers|Storm', regex=True), 'rain_bool'] = True
df_weathercom.loc[df_weathercom['rain_bool'].isna(), 'rain_bool'] = False

A teď chci přidat sloupec, který bude říkat, na kolik dní dopředu je ta předpověď. 
Předpověď získávám večer na další den ráno. Od zítra 6:00 UTC do pozítří 6:00 UTC je to fday 1.
Odečítám tedy od sebe datum, na kdy je předpověď od data, kdy předpověď stahuji.  
Jak narvat dva sloupce do jedné funkce jsem zkopčila zde: https://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe

In [22]:
def forecastdays(date_stamp, time):
    delta = datetime.datetime.date(time) - datetime.datetime.date(date_stamp)
    return float(str(delta)[0])

df_weathercom["wc_fday"] = df_weathercom.apply(lambda x: forecastdays(x["date_stamp"], x["time"]), axis=1)

Ještě ty chytrý názvy.

In [23]:
df_weathercom = df_weathercom.rename(columns={"time": "forecast_date", "location_name": "location"})

A zdroj...

In [24]:
df_weathercom["source"] = "weathercom"

Tohle je zdánlivě nesmyslný řádek, ale opravuje chybu, na kterou jsem přišla až na konci...

In [25]:
df_weathercom["forecast_date"] = df_weathercom["forecast_date"].dt.strftime('%Y-%m-%d')

Ještě nechci data, kde fday je nula, to je mi k prdu

In [26]:
df_weathercom = df_weathercom[df_weathercom["wc_fday"] > 0]

## 4. YR.NO

Norové dávají data v UTC, takže to je ok. Problémek trochu je, že mám proměnlivý date_stamp kvůli změně času, takže nemůžu používat timedelta od date_stamp. Ale nějak si poradím.

In [27]:
df_yrno = pd.read_csv("in/tables/yr_tab.csv")

In [28]:
df_yrno = df_yrno.rename(columns={"air_temperature_C": "yr_temp", "precipitation_amount_mm": "yr_rain"})

In [29]:
df_yrno["date_stamp"] = pd.to_datetime(df_yrno["date_stamp"])
df_yrno = df_yrno[(df_yrno["date_stamp"].dt.time.astype(str) == "19:00:00") | (df_yrno["date_stamp"].dt.time.astype(str) == "20:00:00")]

Asi si nejdříve vytvořím sloupeček yr_fday - tedy na kolikátý den mám předpověď. A pak to budu prostě jenom groupovat podle toho. 

Zde říkám: najdi mi takové řádky, kde "start" je větší než zítra(+1den) 6:00, a zároveň "end" (konec předpovědi) je menší než pozítří (+2dny) v 6:00. 

Podle této formulky jsem upravila zápis tak, aby při splnění podmínky mi vznikl sloupec yr_fday a v něm 1 jako počet dní, na jak dlouho je přepdověď dopředu.   
```df.loc[df['column name'] condition, 'new column name'] = 'value if condition is met'```

In [30]:
df_yrno.loc[(df_yrno["start"] >= (pd.to_datetime(df_yrno["date_stamp"].dt.date + datetime.timedelta(days=1)) + datetime.timedelta(hours=6))) & 
        (df_yrno["end"] <= (pd.to_datetime(df_yrno["date_stamp"].dt.date + datetime.timedelta(days=2)) + datetime.timedelta(hours=6)))
            , ["yr_fday"]] = 1

Asi jsem z toho také mohla udělat funkci. Ale prdím na to, prostě to rozkopíruju pro všech deset dní. Cesta nejmenšího odporu. Trochu z toho dělám SQLko... 😂

In [31]:
df_yrno.loc[(df_yrno["start"] >= (pd.to_datetime(df_yrno["date_stamp"].dt.date + datetime.timedelta(days=2)) + datetime.timedelta(hours=6))) & 
        (df_yrno["end"] <= (pd.to_datetime(df_yrno["date_stamp"].dt.date + datetime.timedelta(days=3)) + datetime.timedelta(hours=6)))
            , ["yr_fday"]] = 2

df_yrno.loc[(df_yrno["start"] >= (pd.to_datetime(df_yrno["date_stamp"].dt.date + datetime.timedelta(days=3)) + datetime.timedelta(hours=6))) & 
        (df_yrno["end"] <= (pd.to_datetime(df_yrno["date_stamp"].dt.date + datetime.timedelta(days=4)) + datetime.timedelta(hours=6)))
            , ["yr_fday"]] = 3

df_yrno.loc[(df_yrno["start"] >= (pd.to_datetime(df_yrno["date_stamp"].dt.date + datetime.timedelta(days=4)) + datetime.timedelta(hours=6))) & 
        (df_yrno["end"] <= (pd.to_datetime(df_yrno["date_stamp"].dt.date + datetime.timedelta(days=5)) + datetime.timedelta(hours=6)))
            , ["yr_fday"]] = 4

df_yrno.loc[(df_yrno["start"] >= (pd.to_datetime(df_yrno["date_stamp"].dt.date + datetime.timedelta(days=5)) + datetime.timedelta(hours=6))) & 
        (df_yrno["end"] <= (pd.to_datetime(df_yrno["date_stamp"].dt.date + datetime.timedelta(days=6)) + datetime.timedelta(hours=6)))
            , ["yr_fday"]] = 5

df_yrno.loc[(df_yrno["start"] >= (pd.to_datetime(df_yrno["date_stamp"].dt.date + datetime.timedelta(days=6)) + datetime.timedelta(hours=6))) & 
        (df_yrno["end"] <= (pd.to_datetime(df_yrno["date_stamp"].dt.date + datetime.timedelta(days=7)) + datetime.timedelta(hours=6)))
            , ["yr_fday"]] = 6

df_yrno.loc[(df_yrno["start"] >= (pd.to_datetime(df_yrno["date_stamp"].dt.date + datetime.timedelta(days=7)) + datetime.timedelta(hours=6))) & 
        (df_yrno["end"] <= (pd.to_datetime(df_yrno["date_stamp"].dt.date + datetime.timedelta(days=8)) + datetime.timedelta(hours=6)))
            , ["yr_fday"]] = 7

df_yrno.loc[(df_yrno["start"] >= (pd.to_datetime(df_yrno["date_stamp"].dt.date + datetime.timedelta(days=8)) + datetime.timedelta(hours=6))) & 
        (df_yrno["end"] <= (pd.to_datetime(df_yrno["date_stamp"].dt.date + datetime.timedelta(days=9)) + datetime.timedelta(hours=6)))
            , ["yr_fday"]] = 8

df_yrno.loc[(df_yrno["start"] >= (pd.to_datetime(df_yrno["date_stamp"].dt.date + datetime.timedelta(days=9)) + datetime.timedelta(hours=6))) & 
        (df_yrno["end"] <= (pd.to_datetime(df_yrno["date_stamp"].dt.date + datetime.timedelta(days=10)) + datetime.timedelta(hours=6)))
            , ["yr_fday"]] = 9

Teď to asi zgrupuju.

viz: https://stackoverflow.com/questions/14529838/apply-multiple-functions-to-multiple-groupby-columns/53096340  
reset_index() mi dá do každého řádku info ze sloupců, podle kterých grupuji - zplacatí mi index v řádcích

In [32]:
df_yrno_g = df_yrno.groupby(['location', 'date_stamp', 'yr_fday', 'source']).agg({'yr_temp':['mean', 'max', 'min'], 
                         'yr_rain':'sum'}).reset_index()

Průšvih je, že abych s tím mohla dál pracovat, tak potřebuji zplacatit index i ve sloupcích. Tomáš mi našel super formulku:
```
df.columns = [' '.join(col).strip() for col in df.columns.values]
```

In [33]:
df_yrno_g.columns = ['_'.join(col).strip('_') for col in df_yrno_g.columns.values]

No a abych to mohla najoinovat, tak ještě potřebuji forecast_date - tedy datum z date_stamp + yr_fday.

In [34]:
def makefdate(dstamp, fday):
    forecast_date = datetime.datetime.date(dstamp) + datetime.timedelta(days = fday)
    return forecast_date
    
df_yrno_g["forecast_date"] = df_yrno_g.apply(lambda x: makefdate(x["date_stamp"], x["yr_fday"]), axis=1)

Abych měla stejný datový typ ve všech tabulkách, potřebuji tento řádek:

In [35]:
df_yrno_g["forecast_date"] = pd.to_datetime(df_yrno_g["forecast_date"]).dt.strftime('%Y-%m-%d')

ještě potřebuji přidat sloupeček, který mi řekne, jestli tam jsou srážky (True/False) pouze kvalitativně. 

In [36]:
df_yrno_g.loc[df_yrno_g["yr_rain_sum"] > 0, "rain_bool"] = True
df_yrno_g.loc[df_yrno_g["yr_rain_sum"] == 0, "rain_bool"] = False

## 5. OPENWEATHER

Tady je největší problém to, že předpověď je od 10 do 10, zatímco čhmú dává data od 6 do 6. A s tím nenadělám nic. 4 hodiny posun už je docela dost...

In [37]:
df_ow = pd.read_csv("in/tables/openweather_tab.csv")

temp_max a temp_min jsou jasný. temp_mean udělám tak, že zprůměruji temp_day, eve, morn, a night. Dávat tam max a min se mi nezdá dobré...

In [38]:
df_ow["temp_mean"] = df_ow[["temp_day", "temp_eve", "temp_morn", "temp_night"]].mean(axis=1)

Teď můžu vykopnout sloupečky, který jsem použila, neb je již nebudu potřebovat.

In [39]:
df_ow = df_ow.drop(columns=["humidity", "temp_day", "temp_eve", "temp_morn", "temp_night"])

Teď chci zjistit fday a forecast_dt převést na forecast_date (pouze datum)

In [40]:
df_ow["fday"] = pd.to_numeric(((pd.to_datetime(df_ow["forecast_dt"]).dt.date) - (pd.to_datetime(df_ow["date_stamp"]).dt.date)).astype("string").str.strip(" days"))

In [41]:
df_ow["forecast_dt"] = pd.to_datetime(df_ow["forecast_dt"]).dt.strftime('%Y-%m-%d')

In [42]:
df_ow = df_ow.rename(columns={"forecast_dt": "forecast_date"})

In [43]:
df_ow["date_stamp"] = pd.to_datetime(df_ow["date_stamp"])

Ještě nechci data, kde fday je nula, to je mi k prdu

In [44]:
df_ow = df_ow[df_ow["fday"] > 0]

In [64]:
df_ow.loc[df_ow["rain"] > 0, "rain_bool"] = True
df_ow.loc[df_ow["rain"].isna(), "rain_bool"] = False

In [65]:
df_ow

,forecast_date,rain,snow,temp_max,temp_min,location,date_stamp,source,temp_mean,fday,rain_bool
0,2021-11-14,7.28,NaN,6.69,4.43,Ústí nad Orlicí,2021-11-08 19:00:00,openweather,5.5725,6,True
3,2021-11-10,NaN,NaN,5.69,1.43,Svratouch,2021-11-08 19:00:00,openweather,2.8275,2,False
4,2021-11-10,NaN,NaN,7.67,0.36,Cheb,2021-11-08 19:00:00,openweather,3.1200,2,False
5,2021-11-15,0.33,0.14,5.02,1.01,Praha-Libuš,2021-11-08 19:00:00,openweather,2.6825,7,True
6,2021-11-09,NaN,NaN,6.42,1.04,Červená u Libavé,2021-11-08 19:00:00,openweather,2.6750,1,False
...,...,...,...,...,...,...,...,...,...,...,...
4475,2021-11-15,NaN,NaN,10.90,3.32,Košetice,2021-11-12 19:00:00,openweather,6.6050,3,False
4476,2021-11-15,NaN,NaN,9.76,3.64,Cheb,2021-11-12 19:00:00,openweather,6.0550,3,False
4477,2021-11-18,1.05,NaN,6.64,2.87,Přibyslav,2021-11-12 19:00:00,openweather,4.8950,6,True
4478,2021-11-17,NaN,NaN,6.44,1.41,Praha-Ruzyně,2021-11-12 19:00:00,openweather,4.7350,5,False


## 6. JOINOVÁNÍ

Nejdřív dám pod sebe data z různých zdrojů a potom na ně najoinuju čhmú. Sice jsem si pojmenovávala sloupce chytře tak, aby byly názvy unikátní, ale naopak teď potřebuji, aby se korespondující sloupce napříč tabulkami jmenovaly stejně.

In [46]:
df_weathercom = df_weathercom.rename(columns={"wc_temp_max": "temp_max", "wc_temp_min": "temp_min", "wc_fday": "fday"})

In [47]:
df_yrno_g = df_yrno_g.rename(columns={"yr_temp_max": "temp_max", "yr_temp_min": "temp_min", "yr_temp_mean": "temp_mean", "yr_rain_sum": "rain", "yr_fday": "fday"})

Teď mohu tabulky spojit pod sebe.

In [48]:
df_forecasts = pd.concat([df_weathercom, df_yrno_g, df_ow])

Pak spojím tabulky ČHMÚ

In [49]:
df_chmi = pd.merge(df_chmi_temp, df_chmi_rain, on = ["date_stamp", "forecast_date", "location"], how = "left")

...a konečně najoinuji chmi na předpovědi

In [50]:
df_joined = pd.merge(df_forecasts, df_chmi, on = ["forecast_date", "location"], how = "left", suffixes=('_f', '_chmi'))

Abych mohla namapovat stanice na mapový podklad, tak přidávám ještě lan a lon.

In [51]:
gps = {'location': ['Cheb', 'Karlovy Vary', 'Přimda', 'Kopisty', 'Tušimice', 'Plzeň-Mikulka', 'Churáňov', 'Milešovka', 'Kocelovice', 'Ústí nad Labem', 'Doksany', 'Praha-Ruzyně', 'Praha-Karlov', 'Praha-Libuš', 'Temelín', 'České Budějovice', 'Praha-Kbely', 'Liberec', 'Jičín', 'Čáslav', 'Košetice', 'Kostelní Myslová', 'Pec pod Sněžkou', 'Pardubice', 'Přibyslav', 'Polom', 'Ústí nad Orlicí', 'Svratouch', 'Náměšť nad Oslavou', 'Dukovany', 'Kuchařovice', 'Luká', 'Brno-Tuřany', 'Šerák', 'Prostějov', 'Červená u Libavé', 'Holešov', 'Ostrava-Mošnov', 'Lysá hora', 'Maruška'],
       'latitude': ['50.0683', '50.2016', '49.6694', '50.544', '50.3765', '49.7645', '49.0683', '50.5549', '49.4672', '50.6833', '50.4587', '50.1003', '50.0691', '50.0077', '49.1975', '48.9519', '50.1232', '50.7697', '50.4393', '49.9407', '49.5735', '49.159', '50.6918', '50.0158', '49.5825', '50.3503', '49.9801', '49.735', '49.1708', '49.0954', '48.8809', '49.6522', '49.153', '50.1874', '49.4525', '49.777', '49.3205', '49.6918', '49.5459', '49.365'], 
       'longitude': ['12.3913', '12.9139', '12.6779', '13.6227', '13.3279', '13.3787', '13.615', '13.9306', '13.8385', '14.041', '14.1699', '14.2555', '14.4276', '14.4467', '14.3421', '14.4697', '14.538', '15.0238', '15.3525', '15.3863', '15.0803', '15.4391', '15.7287', '15.7402', '15.7623', '16.3221', '16.4221', '16.0342', '16.1205', '16.1344', '16.0852', '16.9533', '16.6888', '17.1082', '17.1347', '17.5418', '17.5699', '18.1126', '18.4473', '17.8284']}
df_gps = pd.DataFrame.from_dict(gps, orient="columns")
df_gps["latitude"] = pd.to_numeric(df_gps["latitude"])
df_gps["longitude"] = pd.to_numeric(df_gps["longitude"])

In [52]:
df_joined = pd.merge(df_joined, df_gps, on = ["location"], how = "left")

In [53]:
df_joined.to_csv("out/tables/df_joined.csv", index=False)

## 7. STATISTIKA

In [54]:
df_joined["temp_max_diff"] = df_joined["temp_max"] - df_joined["chmi_temp_max"]
df_joined["temp_min_diff"] = (df_joined["temp_min"] - df_joined["chmi_temp_min"])
df_joined["temp_mean_diff"] = (df_joined["temp_mean"] - df_joined["chmi_temp_avg"])
df_joined["rain_diff"] = (df_joined["rain"] - df_joined["chmi_rain"])
df_joined["temp_max_diff_abs"] = (df_joined["temp_max"] - df_joined["chmi_temp_max"]).abs()
df_joined["temp_min_diff_abs"] = (df_joined["temp_min"] - df_joined["chmi_temp_min"]).abs()
df_joined["temp_mean_diff_abs"] = (df_joined["temp_mean"] - df_joined["chmi_temp_avg"]).abs()
df_joined["rain_diff_abs"] = (df_joined["rain"] - df_joined["chmi_rain"]).abs()

Chci zjistit, jestli se předpověď trefí když řekne, že má (nebo nemá) pršet.  
Když předpověď řekne, že bude pršet a skutečno pršelo, tak je to True, True a předpověď je správná. I v případě False False je předpověď správná - takové výsledky označím jako **rain_match**  
Pak mám **false negative** - to je, když předpověď je False, ale skutečnost True.   
A **false positive** - to je, když je předpověď True, ale skutečnost False. 

In [55]:
df_joined.loc[((df_joined["rain_bool"] == True) & (df_joined["chmi_rain_bool"] == True)) | ((df_joined["rain_bool"] == False) & (df_joined["chmi_rain_bool"] == False)), "rain_match"] = 1
df_joined.loc[(df_joined["rain_bool"] == False) & (df_joined["chmi_rain_bool"] == True), "rain_falsenegative"] = 1
df_joined.loc[(df_joined["rain_bool"] == True) & (df_joined["chmi_rain_bool"] == False), "rain_falsepositive"] = 1

In [56]:
df_grouped1 = df_joined.groupby(["location", "fday", "source"]).agg({'temp_max_diff':['mean', 'median', 'std', 'var', 'count'], 
                                                       'temp_min_diff':['mean', 'median', 'std', 'var', 'count'], 
                                                       'temp_mean_diff':['mean', 'median', 'std', 'var', 'count'], 
                                                       'rain_diff':['mean', 'median', 'std', 'var', 'count'], 
                                                       'temp_max_diff_abs':['mean', 'median', 'std', 'var'], 
                                                       'temp_min_diff_abs':['mean', 'median', 'std', 'var'], 
                                                       'temp_mean_diff_abs':['mean', 'median', 'std', 'var'], 
                                                       'temp_mean_diff_abs':['mean', 'median', 'std', 'var'], 
                                                       'rain_match':'sum', 
                                                       'rain_falsenegative': 'sum', 
                                                       'rain_falsepositive': 'sum',
                                                      }).reset_index()

In [57]:
df_grouped1.columns = ['_'.join(col).strip('_') for col in df_grouped1.columns.values]

In [70]:
df_grouped1["rain_match_percent"] = df_grouped1["rain_match_sum"] / (df_grouped1["rain_match_sum"] + df_grouped1["rain_falsenegative_sum"] + df_grouped1["rain_falsepositive_sum"])
df_grouped1["rain_falsenegative_percent"] = df_grouped1["rain_falsenegative_sum"] / (df_grouped1["rain_match_sum"] + df_grouped1["rain_falsenegative_sum"] + df_grouped1["rain_falsepositive_sum"])
df_grouped1["rain_falsepositive_percent"] = df_grouped1["rain_falsepositive_sum"] / (df_grouped1["rain_match_sum"] + df_grouped1["rain_falsenegative_sum"] + df_grouped1["rain_falsepositive_sum"])

In [71]:
df_grouped1

,location,fday,source,temp_max_diff_mean,temp_max_diff_median,temp_max_diff_std,temp_max_diff_var,temp_max_diff_count,temp_min_diff_mean,temp_min_diff_median,...,temp_mean_diff_abs_mean,temp_mean_diff_abs_median,temp_mean_diff_abs_std,temp_mean_diff_abs_var,rain_match_sum,rain_falsenegative_sum,rain_falsepositive_sum,rain_match_percent,rain_falsenegative_percent,rain_falsepositive_percent
0,Brno-Tuřany,1.0,openweather,-1.250000,-0.85,3.095583,9.582636,12,3.731667,4.195,...,0.799792,0.55125,0.959157,0.919982,0.0,0.0,0.0,NaN,NaN,NaN
1,Brno-Tuřany,1.0,weathercom,-0.052632,0.10,1.835262,3.368187,19,1.568421,1.200,...,NaN,NaN,NaN,NaN,17.0,0.0,1.0,0.944444,0.000000,0.055556
2,Brno-Tuřany,1.0,yrno,-1.450000,-1.35,1.576639,2.485789,20,0.750000,0.750,...,0.787917,0.36250,0.736087,0.541824,18.0,0.0,1.0,0.947368,0.000000,0.052632
3,Brno-Tuřany,2.0,openweather,-2.438182,-0.84,4.272722,18.256156,11,3.489091,4.350,...,1.137273,0.61000,1.440537,2.075147,0.0,0.0,0.0,NaN,NaN,NaN
4,Brno-Tuřany,2.0,weathercom,-0.222222,-0.15,2.130881,4.540654,18,1.533333,1.200,...,NaN,NaN,NaN,NaN,17.0,0.0,0.0,1.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,Šerák,7.0,yrno,-2.300000,-1.95,5.068758,25.692308,14,-1.592857,-0.150,...,2.483929,1.22500,3.349262,11.217558,10.0,1.0,3.0,0.714286,0.071429,0.214286
987,Šerák,8.0,weathercom,2.950000,4.25,5.174676,26.777273,12,2.933333,4.000,...,NaN,NaN,NaN,NaN,9.0,0.0,3.0,0.750000,0.000000,0.250000
988,Šerák,8.0,yrno,-1.661538,0.40,4.957408,24.575897,13,-1.738462,-0.600,...,2.344231,1.07500,3.317381,11.005016,8.0,2.0,3.0,0.615385,0.153846,0.230769
989,Šerák,9.0,weathercom,3.054545,5.20,4.809441,23.130727,11,2.827273,3.300,...,NaN,NaN,NaN,NaN,9.0,0.0,2.0,0.818182,0.000000,0.181818


In [59]:
df_grouped2 = df_joined.groupby(["source", "fday"]).agg({'temp_max_diff':['mean', 'median', 'std', 'var', 'count'], 
                                                       'temp_min_diff':['mean', 'median', 'std', 'var', 'count'], 
                                                       'temp_mean_diff':['mean', 'median', 'std', 'var', 'count'], 
                                                       'rain_diff':['mean', 'median', 'std', 'var', 'count'], 
                                                       'temp_max_diff_abs':['mean', 'median', 'std', 'var'], 
                                                       'temp_min_diff_abs':['mean', 'median', 'std', 'var'], 
                                                       'temp_mean_diff_abs':['mean', 'median', 'std', 'var'], 
                                                       'temp_mean_diff_abs':['mean', 'median', 'std', 'var'], 
                                                       'rain_match':'sum', 
                                                       'rain_falsenegative': 'sum', 
                                                       'rain_falsepositive': 'sum',
                                                      }).reset_index()

In [60]:
df_grouped2.columns = ['_'.join(col).strip('_') for col in df_grouped2.columns.values]

In [61]:
df_grouped1.to_csv("out/tables/df_grouped1.csv", index=False)
df_grouped2.to_csv("out/tables/df_grouped2.csv", index=False)

#### nepodstatné poznámky, kterých se nechci vzdát
Miluju tuhle stránku: https://regex101.com/  
Jak se dostat do sloupce s multiple indexem
```
df.loc[:, [("horni","dolni")]]
```

Jedny hranaté závorky v groupby mi vytvoří Series, dvoje závorky DataFrame, .agg() udělá taky dataframe